In [0]:
from pyspark.sql.functions import explode, col, current_timestamp, coalesce, lower


In [0]:
dbutils.fs.ls("abfss://landing@acmemcd.dfs.core.windows.net/support/dt=2025-11-17")

In [0]:
support_df = spark.read.format("json").load("abfss://landing@acmemcd.dfs.core.windows.net/support")

In [0]:
support_df.printSchema()
display(support_df)

In [0]:
silver_df = support_df.select(
    col("account_id").cast("int").alias("account_id"),
    col("csat").cast("string").alias("csat"),
    col("merge_ref").cast("string").alias("merge_ref"),
    col("priority").cast("string").alias("priority"),
    col("requester_email").cast("string").alias("requester_email"),
    col("text").cast("string").alias("ticket_text"),
    col("ticket_id").cast("string").alias("ticket_id"),
    col("created_at").cast("timestamp").alias("created_at"),
    col("dt").cast("date").alias("dt"),
    current_timestamp().cast("timestamp").alias("ingestion_date")
)
display(silver_df)


In [0]:
invalid_condition = (
    col("account_id").isNull() |
    col("requester_email").isNull() |
    col("ticket_text").isNull() |
    col("ticket_id").isNull() 
)

In [0]:
valid_df = silver_df.filter(~invalid_condition)
display(valid_df)

In [0]:
invalid_df = silver_df.filter(invalid_condition)
display(invalid_df)